*Ryan Harper*

### Non-NN Stocks Predictor:

**INDICATORS:** <br>
<font color=blue>__HOUST__</font> : Housing Starts: Total: New Privately Owned Housing Units Started <br>
<font color=blue>__UNRATENSA__</font>:Civilian Unemployment Rate NSA<br>
<font color=blue>__EMRATIO__</font>: Civilian Employment-Population Ratio<br>
<font color=blue>__UEMPMED__</font>: Median Duration of Unemployment<br>
<font color=blue>__UMCSENT__</font>: University of Michigan: Consumer Sentiment<br>
<font color=blue>__USSLIND__</font>: Leading Index For United States<br>
<font color=blue>__KCFSI__</font>: Kansas City Financial Stress Index<br>
<font color=blue>__IPMAN__</font></font>: Industrial Production: Manufacturing (NAICS)<br>
<font color=blue>__VIXCLS__</font>: CBOE Volatility Index: NSA<br>
<font color=blue>__DGS10__</font>: 10-Year Treasury Constant Maturity Rate<br>

In [645]:
import pandas as pd
import numpy as np
from pandas_datareader import data, wb
import re


# ======== imports and clean csv files ========

# indicators
indicators = pd.read_csv('Indicators_train.csv').iloc[1:].dropna(how='all')
indicators.date = pd.to_datetime(indicators.date)
idicators = indicators.set_index(indicators.date)
for column in indicators:
    if 'dt' in column:
        del indicators[column]
indicators = indicators.set_index(indicators.date)
del indicators['date']

# stock
stock = pd.read_csv('stocks-us-adjClose.csv')
stock.date = pd.to_datetime(stock.date)
stock = stock.set_index(stock.date)
del stock['date']

# sector
sector = pd.read_csv('Company_information.csv', encoding = "ISO-8859-1")
seclist = sector.Sector.unique()
sectorset = []
print('|',end='')
for member in seclist:
    if str(member) != 'nan':
        sectorset.append(member)
        print(member, end='|')
        
# Create dataframes for each sector
utilities = pd.DataFrame(data=stock, columns=sector.loc[sector['Sector'] == 'Utilities']['Symbol'].tolist())
basicmaterials = pd.DataFrame(data=stock, columns=sector.loc[sector['Sector'] == 'Basic Materials']['Symbol'].tolist())
consumer = pd.DataFrame(data=stock, columns=sector.loc[sector['Sector'] == 'Consumer Goods']['Symbol'].tolist())
industrial = pd.DataFrame(data=stock, columns=sector.loc[sector['Sector'] == 'Industrial Goods']['Symbol'].tolist())
healthcare = pd.DataFrame(data=stock, columns=sector.loc[sector['Sector'] == 'Healthcare']['Symbol'].tolist())
services = pd.DataFrame(data=stock, columns=sector.loc[sector['Sector'] == 'Services']['Symbol'].tolist())
financial = pd.DataFrame(data=stock, columns=sector.loc[sector['Sector'] == 'Financial']['Symbol'].tolist())
technology = pd.DataFrame(data=stock, columns=sector.loc[sector['Sector'] == 'Technology']['Symbol'].tolist())
conglomerates = pd.DataFrame(data=stock, columns=sector.loc[sector['Sector'] == 'Conglomerates']['Symbol'].tolist())

# Merges sector dataframes with indicators
indic_utilities = pd.merge(utilities,indicators, how='inner', left_index=True, right_index=True)
indic_basicmaterials = pd.merge(basicmaterials,indicators, how='inner', left_index=True, right_index=True)

|Utilities|Basic Materials|Consumer Goods|Industrial Goods|Healthcare|Services|Financial|Technology|Conglomerates|